In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict

from dolfin import *
import pygmsh
import meshio
import matplotlib.pyplot as plt
from multirat.meshprocessing import meshio2xdmf, xdmf2fenics
from multirat.expressions import sqnorm_cpp_code
from multirat.parameters import symmetric


class ManufacturedPressure:
    def __init__(self, aj):
        self.aj = aj
        
    def expression(self, **kwargs):
        sqnorm_x = sqnorm_cpp_code(None, dim=2)
        return Expression(f"a * {sqnorm_x}", a=Constant(self.aj), **kwargs)
    
    def boundary_value_dirichlet(self):
        return self.aj
    
    def robin_boundary_function(self, alphaj, Kj):
        return self.aj * (1.0 - 2.0 * Kj / alphaj)


from pathlib import Path
from datetime import datetime

def is2dimensional(mesh):
    return all(mesh.points[:, 2] == 0.0)

def meshio2fenics(mesh):
    if is2dimensional(mesh):
        dim = 2
    else:
        dim = 3
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    p = Path(f"/tmp/mesh-{timestamp}")
    p.mkdir()
    meshio2xdmf(mesh, p, dim)
    return xdmf2fenics(p)


with pygmsh.geo.Geometry() as geom:
    geom.add_circle([0.0, 0.0], 1.0, mesh_size=0.2)
    mesh = geom.generate_mesh()
    
    
mesh, _, _ = meshio2fenics(mesh)

In [3]:
params = {}
mockparams = defaultdict(lambda: 1.0, **params)
compartments = ["e", "pa", "pc", "pv"]
V = pressure_functionspace(mesh, 1, compartments)

    
compartments 
a = {
    "e": -1.0,
    "pa": -2.0,
    "pc": 0.0,
    "pv": 1.0,
}
K = symmetric({idx: mockparams[idx] for idx, ai in a.items()}, compartments)
T = symmetric({idx: mockparams[idx] for idx, ai in a.items()}, compartments)

imap = {comp_i: i for i, comp_i in enumerate(compartments)}
p = {idx: ManufacturedPressure(ai) for idx, ai in a.items()}
ph = {idx: project(pi.expression(degree=2), V.sub(imap[idx]).collapse()) for idx, pi in p.items()}
L = {idx: mockparams[idx] for idx in a}
for ph_i in ph.values():
    plot(ph_i, vmin=-1, vmax=1)
    plt.show()

p_bdry = {
    "e": p["e"].robin_boundary_function(L["e"], L["e"]),
    "pa": p["pa"].robin_boundary_function(L["pa"], L["pa"]),
    "pv": p["pv"].boundary_value_dirichlet()
}

boundaries = {
    "e": [RobinBoundary(L["e"], p_bdry["e"], "everywhere")],
    "pa": [RobinBoundary(L["pa"], p_bdry["pa"], "everywhere")],
    "pc": [],
    "pv": [DirichletBoundary(p_bdry["pv"], "everywhere")],
}

NameError: name 'pressure_functionspace' is not defined

In [6]:
from multirat.meshprocessing import Domain
from multirat.parameters import multicompartment_parameters
from multirat.boundary import DirichletBoundary, RobinBoundary
from multirat.multicompartment import solve_pressure, pressure_functionspace, pressure_variational_form

domain = Domain(mesh, None, None)

Path("../results/pressure").resolve()


p = TrialFunctions(V)
q = TestFunction(V)

F = pressure_variational_form(p, q, compartments, K, T)
# P = solve_pressure(domain, V, compartments, boundaries, params)
# store_pressure(results_path, P, compartments, store_xdmf=True)z

# solute_bcs = {
#     "e": [RobinBoundary(, "everywhere")],
#     "pvs_arteries": [DirichletBoundary(Constant(0.0), "everywhere")],
#     "pvs_capillaries": [DirichletBoundary(Constant(0.0), "everywhere")],
#     "pvs_veins": [DirichletBoundary(Constant(0.0), "everywhere")],
# }

# dt = 600  # 10min timestep.
# T = 1 * 3600  # 4h
# N = int(T / dt) + 1
# c_init = Expression(
#     "exp(-(pow(x[0] - c[0], 2) + pow(x[1]-c[1], 2)) / (length * length))",
#     length=Constant(0.5),
#     c=Constant((0.5, 0.0)),
#     degree=2,
# )

# c0 = Function(V)
# assign(c0.sub(0), project(c_init, V.sub(0).collapse()))

# times = np.array([dt * i for i in range(N)])
# C, mass = solve_solute(c0, p, dt, T, domain, V, compartments, solute_bcs, params)

NameError: name 'V' is not defined